# 90. 帰納バイアス入門：なぜCNNは画像を理解できるのか

## 学習目標

このノートブックでは、以下を学びます：

1. **帰納バイアス（Inductive Bias）**とは何か
2. **No Free Lunch定理**と帰納バイアスの必要性
3. **CNNが持つ帰納バイアス**の概要
4. **帰納バイアスと汎化**の関係

## 目次

1. [帰納バイアスとは](#section1)
2. [No Free Lunch定理](#section2)
3. [CNNの帰納バイアス](#section3)
4. [帰納バイアスと汎化性能](#section4)
5. [まとめ](#summary)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyBboxPatch
import japanize_matplotlib

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

<a id="section1"></a>
## 1. 帰納バイアスとは

### 定義

**帰納バイアス（Inductive Bias）**とは：

> 学習アルゴリズムが、見たことのないデータに対して予測を行う際に用いる**仮定・制約**のこと

### なぜ必要なのか？

有限のデータから無限の可能性のある関数を学習するためには、何らかの「事前の仮定」が必要です。

In [ ]:
def visualize_inductive_bias_concept():
    """帰納バイアスの概念を可視化"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # データ点
    np.random.seed(42)
    x_data = np.array([1, 2, 3, 5, 6])
    y_data = np.array([2.1, 4.2, 5.8, 10.1, 12.0])
    
    x_plot = np.linspace(0, 7, 100)
    
    # 1. データのみ（帰納バイアスなし）
    ax = axes[0]
    ax.scatter(x_data, y_data, c='blue', s=100, zorder=5, label='観測データ')
    ax.set_title('データのみ\n（帰納バイアスなし）', fontsize=14)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.text(4, 3, 'x=4での予測は？\n無限の可能性...', fontsize=11, 
           ha='center', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
    ax.axvline(x=4, color='red', linestyle='--', alpha=0.5)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, 7)
    ax.set_ylim(0, 15)
    
    # 2. 線形モデル（「データは直線に従う」という帰納バイアス）
    ax = axes[1]
    ax.scatter(x_data, y_data, c='blue', s=100, zorder=5)
    
    # 線形フィット
    coeffs = np.polyfit(x_data, y_data, 1)
    y_linear = np.polyval(coeffs, x_plot)
    ax.plot(x_plot, y_linear, 'g-', linewidth=2, label='線形フィット')
    
    # x=4での予測
    y_pred = np.polyval(coeffs, 4)
    ax.scatter([4], [y_pred], c='red', s=150, marker='*', zorder=6)
    
    ax.set_title('線形モデル\n「直線に従う」という仮定', fontsize=14)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.text(4, y_pred + 1.5, f'予測: {y_pred:.1f}', fontsize=11, ha='center', color='red')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, 7)
    ax.set_ylim(0, 15)
    
    # 3. 高次多項式（異なる帰納バイアス）
    ax = axes[2]
    ax.scatter(x_data, y_data, c='blue', s=100, zorder=5)
    
    # 4次多項式フィット
    coeffs_poly = np.polyfit(x_data, y_data, 4)
    y_poly = np.polyval(coeffs_poly, x_plot)
    ax.plot(x_plot, y_poly, 'r-', linewidth=2, label='4次多項式')
    
    # x=4での予測
    y_pred_poly = np.polyval(coeffs_poly, 4)
    ax.scatter([4], [y_pred_poly], c='red', s=150, marker='*', zorder=6)
    
    ax.set_title('高次多項式\n「複雑な曲線」という仮定', fontsize=14)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.text(4, y_pred_poly - 2, f'予測: {y_pred_poly:.1f}', fontsize=11, ha='center', color='red')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, 7)
    ax.set_ylim(0, 15)
    
    plt.suptitle('帰納バイアス：異なる仮定 → 異なる予測', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print(f"線形モデルの x=4 での予測: {y_pred:.2f}")
    print(f"4次多項式の x=4 での予測: {y_pred_poly:.2f}")
    print("\n→ 同じデータでも、帰納バイアスによって予測が異なる！")

visualize_inductive_bias_concept()

### 帰納バイアスの例

| モデル | 帰納バイアス |
|--------|------------|
| 線形回帰 | データは線形関係に従う |
| 決定木 | 特徴空間は軸に平行な境界で分割できる |
| k-NN | 近いデータは同じクラスに属する |
| **CNN** | 画像には局所パターンがあり、それは位置に依存しない |

<a id="section2"></a>
## 2. No Free Lunch定理

### 定理の内容

**No Free Lunch定理**（Wolpert & Macready, 1997）：

> すべての可能な問題に対して平均すると、どの学習アルゴリズムも同じ性能を示す

つまり、「万能のアルゴリズム」は存在しない！

In [ ]:
def visualize_no_free_lunch():
    """No Free Lunch定理の直感的説明"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    np.random.seed(42)
    
    # 問題1: 線形データ
    ax = axes[0]
    x1 = np.linspace(0, 5, 20)
    y1 = 2 * x1 + 1 + np.random.normal(0, 0.5, 20)
    
    ax.scatter(x1, y1, c='blue', s=50)
    
    # 線形フィット
    coeffs = np.polyfit(x1, y1, 1)
    ax.plot(x1, np.polyval(coeffs, x1), 'g-', linewidth=2, label='線形モデル ✓')
    
    ax.set_title('問題1: 線形関係\n線形モデルが良い', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 問題2: 周期的データ
    ax = axes[1]
    x2 = np.linspace(0, 4*np.pi, 40)
    y2 = np.sin(x2) + np.random.normal(0, 0.1, 40)
    
    ax.scatter(x2, y2, c='blue', s=50)
    
    # 線形は失敗
    coeffs = np.polyfit(x2, y2, 1)
    ax.plot(x2, np.polyval(coeffs, x2), 'r--', linewidth=2, alpha=0.5, label='線形モデル ✗')
    
    # sinが正解
    ax.plot(x2, np.sin(x2), 'g-', linewidth=2, label='周期モデル ✓')
    
    ax.set_title('問題2: 周期的関係\n線形モデルは失敗', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 問題3: ランダムデータ
    ax = axes[2]
    x3 = np.random.rand(30) * 5
    y3 = np.random.rand(30) * 5
    
    ax.scatter(x3, y3, c='blue', s=50)
    
    ax.set_title('問題3: ランダム\nどのモデルも同じ（予測不可能）', fontsize=14)
    ax.grid(True, alpha=0.3)
    
    plt.suptitle('No Free Lunch: 問題に合った帰納バイアスが必要', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_no_free_lunch()

### 定理の含意

1. **特定の問題には、それに適した帰納バイアスを持つアルゴリズムが必要**
2. **帰納バイアスが問題に合っていれば、少ないデータで良い汎化が可能**
3. **帰納バイアスが問題に合っていなければ、いくらデータがあっても性能が出ない**

→ CNNが画像認識に優れているのは、画像の性質に合った帰納バイアスを持っているから！

<a id="section3"></a>
## 3. CNNの帰納バイアス

CNNは主に3つの帰納バイアスを持っています：

1. **局所性（Locality）**: 近くのピクセルは関連性が高い
2. **平行移動等変性（Translation Equivariance）**: 同じパターンは画像のどこにあっても同じ方法で処理される
3. **階層性（Hierarchy）**: 単純な特徴から複雑な特徴を段階的に構築

In [ ]:
def visualize_cnn_inductive_biases():
    """CNNの3つの帰納バイアスを可視化"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # 1. 局所性
    ax = axes[0]
    
    # 7x7グリッド
    for i in range(8):
        ax.axhline(y=i, color='lightgray', linewidth=0.5)
        ax.axvline(x=i, color='lightgray', linewidth=0.5)
    
    # 中心ピクセル
    rect = Rectangle((3, 3), 1, 1, linewidth=2, edgecolor='red', facecolor='red', alpha=0.5)
    ax.add_patch(rect)
    
    # 局所的な関係（近傍）
    for i in range(2, 5):
        for j in range(2, 5):
            if not (i == 3 and j == 3):
                rect = Rectangle((j, i), 1, 1, linewidth=1, 
                                 edgecolor='blue', facecolor='lightblue', alpha=0.3)
                ax.add_patch(rect)
    
    # 遠いピクセル
    rect = Rectangle((0, 0), 1, 1, linewidth=1, 
                     edgecolor='gray', facecolor='lightgray', alpha=0.3)
    ax.add_patch(rect)
    
    ax.annotate('強い関係', xy=(3.5, 3.5), xytext=(5.5, 5.5),
               arrowprops=dict(arrowstyle='->', color='blue', lw=2))
    ax.annotate('弱い関係', xy=(3.5, 3.5), xytext=(0.5, 0.5),
               arrowprops=dict(arrowstyle='->', color='gray', lw=1, linestyle='--'))
    
    ax.set_xlim(-0.5, 7.5)
    ax.set_ylim(-0.5, 7.5)
    ax.set_aspect('equal')
    ax.invert_yaxis()
    ax.set_title('1. 局所性\n「近くのピクセルは関連性が高い」', fontsize=14, fontweight='bold')
    ax.axis('off')
    
    # 2. 平行移動等変性
    ax = axes[1]
    
    # 2つの画像（同じパターンが異なる位置）
    img1 = np.zeros((7, 7))
    img2 = np.zeros((7, 7))
    
    # パターン（L字）
    pattern = np.array([[1, 0], [1, 0], [1, 1]])
    
    img1[1:4, 1:3] = pattern
    img2[3:6, 4:6] = pattern
    
    # 上下に並べて表示
    combined = np.vstack([img1, np.ones((1, 7)) * 0.5, img2])
    ax.imshow(combined, cmap='Blues', vmin=0, vmax=1)
    
    ax.axhline(y=7, color='black', linewidth=2)
    
    ax.text(3.5, -0.5, '画像1', fontsize=11, ha='center')
    ax.text(3.5, 8.5, '画像2', fontsize=11, ha='center')
    ax.text(3.5, 15, '同じカーネルで\n同じ特徴を検出', fontsize=11, ha='center',
           bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
    
    ax.set_title('2. 平行移動等変性\n「同じパターンは同じ方法で処理」', fontsize=14, fontweight='bold')
    ax.axis('off')
    
    # 3. 階層性
    ax = axes[2]
    
    # 階層図
    levels = ['ピクセル', 'エッジ', 'テクスチャ', 'パーツ', 'オブジェクト']
    y_positions = [0.1, 0.3, 0.5, 0.7, 0.9]
    
    for y, level in zip(y_positions, levels):
        box = FancyBboxPatch((0.3, y - 0.05), 0.4, 0.08, 
                             boxstyle="round,pad=0.02",
                             facecolor='lightgreen', edgecolor='green',
                             transform=ax.transAxes)
        ax.add_patch(box)
        ax.text(0.5, y, level, fontsize=12, ha='center', va='center',
               transform=ax.transAxes)
    
    # 矢印
    for i in range(len(levels) - 1):
        ax.annotate('', xy=(0.5, y_positions[i+1] - 0.07), 
                   xytext=(0.5, y_positions[i] + 0.05),
                   arrowprops=dict(arrowstyle='->', color='black', lw=1.5),
                   transform=ax.transAxes)
    
    ax.text(0.85, 0.5, '層を重ねて\n抽象度UP', fontsize=11, 
           transform=ax.transAxes, ha='center', va='center',
           bbox=dict(boxstyle='round', facecolor='lightyellow'))
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_title('3. 階層性\n「単純→複雑を段階的に構築」', fontsize=14, fontweight='bold')
    ax.axis('off')
    
    plt.suptitle('CNNの3つの帰納バイアス', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_cnn_inductive_biases()

In [ ]:
def explain_cnn_biases_in_detail():
    """CNNの帰納バイアスの詳細説明"""
    print("="*70)
    print("CNNの帰納バイアス：詳細")
    print("="*70)
    
    biases = [
        ("1. 局所性 (Locality)",
         "- 畳み込みカーネルは局所的な領域のみを見る",
         "- 遠く離れたピクセルは直接的には関係しないと仮定",
         "- 受容野を通じて間接的に大域的な情報を得る",
         "→ 画像では近くのピクセルが同じオブジェクトに属することが多い"),
        
        ("2. 平行移動等変性 (Translation Equivariance)",
         "- 同じカーネルを画像全体でスライドさせる（重み共有）",
         "- パターンが画像のどこにあっても同じ応答",
         "- 入力がシフト → 出力も同じだけシフト",
         "→ 猫は画像の左にいても右にいても『猫』"),
        
        ("3. 階層性 (Hierarchy)",
         "- 層を重ねることで受容野が拡大",
         "- 浅い層: 低レベル特徴（エッジ、色）",
         "- 深い層: 高レベル特徴（オブジェクト）",
         "→ 視覚システムの階層構造に類似")
    ]
    
    for name, *details in biases:
        print(f"\n{name}")
        print("-" * 50)
        for detail in details:
            print(f"  {detail}")

explain_cnn_biases_in_detail()

### これらの帰納バイアスが画像に適している理由

自然画像には以下の性質があり、CNNの帰納バイアスと一致します：

1. **空間的局所性**: 近くのピクセルは同じオブジェクトに属することが多い
2. **定常性**: 同じパターン（エッジ、テクスチャ）は画像のどこにでも現れうる
3. **構成性**: 複雑な構造は単純な構造の組み合わせから成る

<a id="section4"></a>
## 4. 帰納バイアスと汎化性能

### 帰納バイアスの強さとトレードオフ

- **強い帰納バイアス**: 少ないデータで良い汎化、ただし問題に合わないと性能低下
- **弱い帰納バイアス**: 多くのデータが必要、ただしより多様な問題に対応可能

In [ ]:
def visualize_bias_variance_tradeoff():
    """帰納バイアスとバイアス-バリアンストレードオフ"""
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # 1. データ量と性能の関係
    ax = axes[0]
    
    data_sizes = np.logspace(1, 6, 50)
    
    # 強い帰納バイアス（CNN風）
    strong_bias = 0.95 - 0.7 * np.exp(-data_sizes / 1000)
    
    # 弱い帰納バイアス（MLP風）
    weak_bias = 0.95 - 0.8 * np.exp(-data_sizes / 50000)
    
    ax.semilogx(data_sizes, strong_bias, 'b-', linewidth=2, label='強い帰納バイアス（CNN）')
    ax.semilogx(data_sizes, weak_bias, 'r-', linewidth=2, label='弱い帰納バイアス（MLP）')
    
    ax.axhline(y=0.95, color='green', linestyle='--', alpha=0.5, label='理論的最大性能')
    
    ax.fill_between(data_sizes, strong_bias, weak_bias, 
                   where=strong_bias > weak_bias, alpha=0.2, color='blue',
                   label='CNNが有利な領域')
    ax.fill_between(data_sizes, strong_bias, weak_bias, 
                   where=strong_bias < weak_bias, alpha=0.2, color='red',
                   label='MLPが有利な領域')
    
    ax.set_xlabel('データ量', fontsize=12)
    ax.set_ylabel('テスト精度', fontsize=12)
    ax.set_title('データ量 vs 汎化性能', fontsize=14, fontweight='bold')
    ax.legend(loc='lower right', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0, 1)
    
    # 2. バイアス-バリアンストレードオフ
    ax = axes[1]
    
    complexity = np.linspace(0, 10, 100)
    
    bias_squared = 0.8 * np.exp(-complexity)
    variance = 0.05 * complexity ** 1.5
    total_error = bias_squared + variance
    
    ax.plot(complexity, bias_squared, 'b-', linewidth=2, label='バイアス²')
    ax.plot(complexity, variance, 'r-', linewidth=2, label='バリアンス')
    ax.plot(complexity, total_error, 'k-', linewidth=2, label='総誤差')
    
    # 最適点
    optimal_idx = np.argmin(total_error)
    ax.axvline(x=complexity[optimal_idx], color='green', linestyle='--', alpha=0.5)
    ax.scatter([complexity[optimal_idx]], [total_error[optimal_idx]], 
              c='green', s=100, zorder=5, label='最適な複雑さ')
    
    ax.set_xlabel('モデルの複雑さ（帰納バイアスの弱さ）', fontsize=12)
    ax.set_ylabel('誤差', fontsize=12)
    ax.set_title('バイアス-バリアンストレードオフ', fontsize=14, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

visualize_bias_variance_tradeoff()

In [ ]:
def compare_model_parameters():
    """モデルのパラメータ効率比較"""
    print("="*70)
    print("帰納バイアスとパラメータ効率")
    print("="*70)
    
    # 32x32x3の入力、64チャンネル出力の場合
    input_size = 32 * 32 * 3  # = 3072
    output_channels = 64
    
    # MLP（全結合）
    # 32x32x3 → 32x32x64 の全結合
    fc_params = input_size * (32 * 32 * output_channels)
    
    # CNN（3x3カーネル）
    kernel_size = 3
    input_channels = 3
    cnn_params = kernel_size * kernel_size * input_channels * output_channels + output_channels  # +bias
    
    print(f"\n入力: 32×32×3 = {input_size}ピクセル")
    print(f"出力: 32×32×64 = {32*32*64}値")
    
    print(f"\n【全結合層（MLP）】")
    print(f"  パラメータ数: {fc_params:,}")
    print(f"  帰納バイアス: なし（全ピクセルが全出力に接続）")
    
    print(f"\n【畳み込み層（CNN）】")
    print(f"  パラメータ数: {cnn_params:,}")
    print(f"  帰納バイアス: 局所性 + 重み共有")
    
    print(f"\n【比較】")
    print(f"  パラメータ削減率: {fc_params / cnn_params:,.0f}倍")
    print(f"  → 帰納バイアスにより、{fc_params / cnn_params:,.0f}倍少ないパラメータで同等の処理！")

compare_model_parameters()

<a id="summary"></a>
## 5. まとめ

### 学んだこと

1. **帰納バイアスとは**
   - 学習アルゴリズムが持つ事前の仮定・制約
   - 有限データから汎化するために必要

2. **No Free Lunch定理**
   - 万能のアルゴリズムは存在しない
   - 問題に合った帰納バイアスが重要

3. **CNNの帰納バイアス**
   - **局所性**: 近くのピクセルは関連性が高い
   - **平行移動等変性**: 同じパターンは同じ方法で処理
   - **階層性**: 単純→複雑を段階的に構築

4. **帰納バイアスの効果**
   - パラメータ効率の向上（数千倍の削減）
   - 少ないデータでの良好な汎化
   - 画像という問題領域に特化した設計

In [ ]:
def summary_diagram():
    """まとめの図"""
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # 中央: 帰納バイアス
    ax.add_patch(plt.Circle((0.5, 0.5), 0.12, color='gold', alpha=0.7))
    ax.text(0.5, 0.5, '帰納\nバイアス', fontsize=14, ha='center', va='center',
           fontweight='bold')
    
    # 上: 画像の性質
    ax.add_patch(FancyBboxPatch((0.3, 0.75), 0.4, 0.12, 
                                boxstyle="round,pad=0.02",
                                facecolor='lightblue', edgecolor='blue'))
    ax.text(0.5, 0.81, '画像の性質', fontsize=12, ha='center', fontweight='bold')
    ax.text(0.5, 0.77, '局所相関・定常性・構成性', fontsize=10, ha='center')
    
    # 下: 汎化性能
    ax.add_patch(FancyBboxPatch((0.3, 0.15), 0.4, 0.12, 
                                boxstyle="round,pad=0.02",
                                facecolor='lightgreen', edgecolor='green'))
    ax.text(0.5, 0.21, '汎化性能', fontsize=12, ha='center', fontweight='bold')
    ax.text(0.5, 0.17, '少ないデータで高精度', fontsize=10, ha='center')
    
    # 左: CNNの構造
    ax.add_patch(FancyBboxPatch((0.05, 0.4), 0.2, 0.2, 
                                boxstyle="round,pad=0.02",
                                facecolor='lightyellow', edgecolor='orange'))
    ax.text(0.15, 0.55, 'CNNの構造', fontsize=11, ha='center', fontweight='bold')
    ax.text(0.15, 0.47, '・局所カーネル', fontsize=9, ha='center')
    ax.text(0.15, 0.43, '・重み共有', fontsize=9, ha='center')
    
    # 右: パラメータ効率
    ax.add_patch(FancyBboxPatch((0.75, 0.4), 0.2, 0.2, 
                                boxstyle="round,pad=0.02",
                                facecolor='lavender', edgecolor='purple'))
    ax.text(0.85, 0.55, 'パラメータ効率', fontsize=11, ha='center', fontweight='bold')
    ax.text(0.85, 0.47, 'MLP比で', fontsize=9, ha='center')
    ax.text(0.85, 0.43, '数千倍削減', fontsize=9, ha='center')
    
    # 矢印
    ax.annotate('', xy=(0.5, 0.62), xytext=(0.5, 0.75),
               arrowprops=dict(arrowstyle='->', color='blue', lw=2))
    ax.annotate('', xy=(0.5, 0.38), xytext=(0.5, 0.27),
               arrowprops=dict(arrowstyle='<-', color='green', lw=2))
    ax.annotate('', xy=(0.38, 0.5), xytext=(0.25, 0.5),
               arrowprops=dict(arrowstyle='<-', color='orange', lw=2))
    ax.annotate('', xy=(0.62, 0.5), xytext=(0.75, 0.5),
               arrowprops=dict(arrowstyle='->', color='purple', lw=2))
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('帰納バイアス：画像の性質とCNN設計の橋渡し', fontsize=16, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

summary_diagram()

### 次のノートブック

次のノートブックでは、CNNの重要な帰納バイアスである**重み共有（Weight Sharing）**について詳しく学びます。

- 重み共有の数学的定義
- なぜパラメータが削減されるか
- 重み共有がもたらす平行移動等変性